In [ ]:
from pipeline_builder import DockerMLPipelineBuilder

builder = DockerMLPipelineBuilder(
    image="chand1012/tensorflow:2.5.0",
    source_code_dir="src",
    requirements_file="src/requirements.txt",
)

In [ ]:
# Setup Local Directories for training artifacts

import os
output_path = "output"
os.makedirs(output_path, exist_ok=True)
abs_output_path = os.path.abspath(output_path)

data_path = "data"
os.makedirs(data_path, exist_ok=True)
abs_data_path = os.path.abspath(data_path)

In [ ]:
builder.add_preprocessing_stage(
    command="python preprocessing/process.py",
    arguments=[
        "--output_path", "/data",
        "--fraction", "0.5"
    ],
    volumes=[f"{abs_data_path}:/data"]
)

In [ ]:
builder.add_training_stage(
    command="python training/train.py",
    arguments=[
        "--input_path", "/data/training_data.npz", 
        "--output_path", "/output"
    ],
    volumes=[f"{abs_output_path}:/output", f"{abs_data_path}:/data"],
)

In [ ]:
builder.add_serving_stage(
    command="python serving/serve.py",
    arguments=["--model_path", "/output/cnn_model.h5"],
    volumes=[f"{abs_output_path}:/output"]
)

In [ ]:
builder.build_and_run_pipeline()